In [84]:
# Librairies principaux.
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Autres.
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import svm
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

# Scalling.
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [38]:
silicon_valley = pd.read_csv("silicon_valley.csv")
silicon_valley = silicon_valley.rename(columns=
{
      'Unnamed: 0'         : 'ID logement', 
      'housing_median_age' : 'age moyen logement', 
      'total_rooms'        : 'total pieces', 
      'total_bedrooms'     : 'total chambres', 
      'population'         : 'nb personne logement', 
      'households'         : 'nb famille logement', 
      'median_income'      : 'salaire median logement',
      'median_house_value' : 'valeur moyenne logement',
      'ocean_proximity'    : 'proximation Ocean'
})

In [39]:
silicon_valley.head()

,ID logement,longitude,latitude,age moyen logement,total pieces,total chambres,nb personne logement,nb famille logement,salaire median logement,valeur moyenne logement,proximation Ocean
0,2072,-119.84,36.77,6.0,1853.0,473.0,1397.0,417.0,1.4817,72000.0,INLAND
1,10600,-117.80,33.68,8.0,2032.0,349.0,862.0,340.0,6.9133,274100.0,<1H OCEAN
2,2494,-120.19,36.60,25.0,875.0,214.0,931.0,214.0,1.5536,58300.0,INLAND
3,4284,-118.32,34.10,31.0,622.0,229.0,597.0,227.0,1.5284,200000.0,<1H OCEAN
4,16541,-121.23,37.79,21.0,1922.0,373.0,1130.0,372.0,4.0815,117900.0,INLAND


In [40]:
# Séparation des features & la Target
X = silicon_valley.drop('salaire median logement', axis='columns')
Y = silicon_valley["salaire median logement"]

In [80]:
# Classement des colonnes dans des listes.
target         =  ["salaire median logement"]
drop           =  ["ID logement"]
num_manquantes =  ["total chambres"]
cat            =  ["proximation Ocean"]
passthrough    =  []
text           =  []
cat_manquantes =  []

scaling = [
    "total pieces", 
    "nb personne logement", 
    "longitude", 
    "latitude", 
    "age moyen logement",
    "nb famille logement", 
    'valeur moyenne logement'
]

In [81]:
# Liste contenant toutes les colonnes
all_cols = [
    target,
    drop,
    passthrough,
    text,
    num_manquantes,
    cat_manquantes,
    cat,
    scaling
]

In [82]:
# Vérification (fautes ortographes, oublie de colonnes etc...)
def check_work(liste_all_listes, data):
    set_nos_cols = set()
    for liste in liste_all_listes:
        for col in liste:
            if col in set_nos_cols:
                print(f"Warning : La colonne '{col}' est déja présente !")
            set_nos_cols.add(col)         
    set_colonnes_originales = set(data.columns)
    mal_écrites = set_nos_cols - set_colonnes_originales
    col_manquantes = set_colonnes_originales - set_nos_cols
    print(f"Des colonnes sont manquantes :  {col_manquantes}")
    print(f"Des colonnes sont mal écrites : {mal_écrites}")

In [83]:
# exécution de la fonction de vérification.
check_work(liste_all_listes=all_cols, data=silicon_valley)

Des colonnes sont manquantes :  set()
Des colonnes sont mal écrites : set()


## PREPROCESSING

In [90]:
# Importer les utilitaires sklearn pour faire ça proprement
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder

In [93]:
# Preprocessing Complet.
preprocessing = make_column_transformer(
    (OneHotEncoder(handle_unknown="ignore"),  cat),
    (KNNImputer(n_neighbors=1),               num_manquantes),
    ("passthrough",                           passthrough),
    ("drop",                                  drop)      
)

## PipeLine Complète

In [94]:
# Créer une Pipeline
from sklearn.pipeline import Pipeline

In [95]:
# Les Algorithmes de Machine Learning
from sklearn.linear_model import LinearRegression

In [96]:
# Random Forest Modele
linear_regression_pipeline = Pipeline([
    ('Preprocessing',     preprocessing),
    ('LinearRegression',  LinearRegression()),
])

In [99]:
# linear_regression_pipeline.fit(X,Y)
# predictions_forest = linear_regression_pipeline.predict(X)

### GRID SEARCH : Recherche de la Meilleure Pipeline

In [97]:
from sklearn.model_selection import GridSearchCV

#### CrossValidation

In [27]:
cross_validation_folds = KFold(n_splits=5, shuffle=True, random_state=777)

In [ ]:
linear_regression = {}

# pipeline knn
linear_regression["pipeline"] = Pipeline([
                            ('Preprocessing',    preprocessing),
                            ('LinearRegression', LinearRegression()),
])

# Grille pour la pipeline KNN
linear_regression["hyperparamètres"] = {}
linear_regression["hyperparamètres"]["choisir_param_linear_regression"] = [1, 3, 5, 7, 9, 13, 17, 21, 27, 29]
linear_regression["hyperparamètres"]["choisir_param_linear_regression"] = ["uniform", "distance"]

# gridsearch pour la pipeline KNN
linear_regression["gridsearch"] = GridSearchCV(
                                     estimator  = linear_regression["pipeline"],
                                     param_grid = knn["hyperparamètres"],
                                     scoring    = 'balanced_accuracy', 
                                     cv         = cross_validation_folds
                                   )

In [ ]:
knn["gridsearch"].fit(X, Y);
knn["gridsearch"].best_params_;
knn["gridsearch"].best_score_;

## PipeLine Final

In [ ]:
pipeline_final = Pipeline([
    ('preprocessing',    preprocessing), 
    ('LinearRegression', LinearRegression(param_linear_regression= 3, param_linear_regression="distance"))
])

In [ ]:
# Entrainement sur toutes les données
pipeline_final.fit(X, Y);

In [ ]:
# Pipeline : Les prédictions finales (manque le dataset_test)
predict = pipeline_final.predict(df_test)